In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import json
from utils import plotly_utils
from collections import defaultdict
import pandas as pd
from collections import defaultdict
import random

In [14]:
orig = json.load(open('layouts/test/yahoo_1/dimensions.json', 'r'))
archive = json.load(open('layouts/test/yahoo_1_archive/dimensions.json', 'r'))

diff = defaultdict(list)
t = 0
for elem, dimem in orig['dimensions'].items():
    if sum(dimem.values()) == 0:
        continue
    if elem not in archive['dimensions']:
        t += 1
        # print(elem)
        continue
    adimem = archive['dimensions'][elem]
    
    for k, v in dimem.items():
        if v == 0 or adimem[k] == 0:
            continue
        diff[k].append(v - adimem[k])
        # if k == 'top' and v - adimem[k] > 0:
        #     print(elem)

print({k: len(v) for k, v in diff.items()})
print(t)
df = pd.DataFrame({k: pd.Series(v) for k, v in diff.items()})
plotly_utils.plot_CDF(df)


{'width': 224, 'height': 203, 'right': 238, 'bottom': 238, 'y': 233, 'top': 233, 'x': 217, 'left': 217}
114


In [4]:
from bs4 import BeautifulSoup
from lxml import etree

html1 = open('layouts/test/cnn_3_archive/index.html', 'r').read()
archive_soup = BeautifulSoup(html1, 'lxml')

html2 = open('layouts/test/cnn_3/index.html', 'r').read()
live_soup = BeautifulSoup(html2, 'lxml')

archive_etree = etree.HTML(html1)
live_etree = etree.HTML(html2)
element = archive_etree.xpath("/html/head[1]")[0]
xpath = "/html/head[1]/*"
element = live_etree.xpath(xpath)

# element
etree.tostring(element[120])

b'<script src="/optimizelyjs/131788053.js"/>'

In [40]:
def get_elem_id(elem):
    eid = elem.name.upper()
    if 'id' in elem.attrs:
        eid += "#"
        eid += elem.attrs['id'] if isinstance(elem.attrs['id'], str) else ' '.join(elem.attrs['id'])
    if 'class' in elem.attrs:
        eid += "."
        eid += elem.attrs['class'] if isinstance(elem.attrs['class'], str) else ' '.join(elem.attrs['class'])
    if elem.name == "a" and 'href' in elem.attrs:
        eid += f"#href:{elem.attrs['href']}"
    return eid

### Plot pixel similarity CDF

In [4]:
bcarta = json.load(open('results_v3/carta/img_diff.json', 'r'))
top_sites = json.load(open('results_v3/topsites/img_diff.json', 'r'))
carta = [float(c['similarity']) for c in carta if c['similarity'] > 0]
top_sites = [float(c['similarity']) for c in top_sites if c['similarity'] > 0]
print(len(carta), len(top_sites))
d = {'carta': carta, 'topsites': top_sites}
df = pd.DataFrame({k: pd.Series(v) for k, v in d.items()})
plotly_utils.plot_CDF(df, xtitle='Pixel similarity', ytitle='CDF across pages')

87 66


In [9]:
replay_pixel_diff = json.load(open('layouts/replay_pixel_diff.json', 'r'))
mins = [m['min_simi'] for m in replay_pixel_diff if m['min_simi'] is not None]
maxs = [m['max_simi'] for m in replay_pixel_diff if m['max_simi'] is not None]
print(len(maxs))
print(len([m for m in maxs if m >= 1]), len([m for m in mins if m < 1]))
df = pd.DataFrame({'min': mins, 'max': maxs})
plotly_utils.plot_CDF(df, xtitle='Pixel similarity')

78
77 54


In [5]:
meta_t = json.load(open('results_v3/topsites/metadata.json', 'r'))
meta_c = json.load(open('results_v3/carta/metadata.json', 'r'))

meta_t = [m for m in meta_t.values() if m['ts'] != ""]
meta_c = [m for m in meta_c.values() if m['ts'] != ""]

metas = meta_t + meta_c
metas = random.sample(metas, 100)
json.dump(metas, open('recording/sampled_metadata.json', 'w+'), indent=2)